<a href="https://colab.research.google.com/github/lala991204/ML-self-study/blob/master/9_4__Collaborative_filtering_of_potential_factors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 경사하강을 이용한 행렬 분해

In [1]:
import numpy as np

# 원본 행렬 R 생성, 분해 행렬 P,Q 초기화하고 잠재요인 차원 K=3으로 설정
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN],
             [np.NaN, 5, np.NaN, 3, 1],
             [np.NaN, np.NaN, 3, 4, 4],
             [5, 2, 1, 2, np.NaN]])
num_users, num_items = R.shape
K=3

# P, Q 행렬의 크기를 지정하고 정규분포를 가진 random한 값으로 입력
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))   # scale: 표준편차, size: 샘플의 사이즈
Q = np.random.normal(scale=1./K, size=(num_items, K))

In [3]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두 개의 분해된 행렬 P, Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)

    # 실제 R 행렬에서 Null이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]

    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)

    return rmse

In [6]:
# R > 0인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장
non_zeros = [(i, j, R[i, j]) for i in range(num_users) for j in range(num_items)
       if R[i, j] > 0]

steps = 1000
learning_rate = 0.01
r_lambda = 0.01

# SGD 기법으로 P, Q 행렬을 계속 업데이트함
for step in range(steps):
    for i, j, r in non_zeros:
        # 실제 값과 예측 값의 차이인 오류 값 구함
        eij = r - np.dot(P[i, :], Q[j, :].T)

        # Regularization을 반영한 SGD 업데이트 공식 적용
        P[i, :] = P[i, :] + learning_rate*(eij * Q[j, :] - r_lambda*P[i, :])
        Q[j, :] = Q[j, :] + learning_rate*(eij * P[i, :] - r_lambda*Q[j, :])

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 50) == 0:
        print("### iteration step : ", step, " rmse : ", rmse)

### iteration step :  0  rmse :  3.18918148042182
### iteration step :  50  rmse :  0.45402815018032405
### iteration step :  100  rmse :  0.15115898740988784
### iteration step :  150  rmse :  0.07269965774474765
### iteration step :  200  rmse :  0.04246278530171519
### iteration step :  250  rmse :  0.028877999722529208
### iteration step :  300  rmse :  0.022445550468806877
### iteration step :  350  rmse :  0.019411195442598027
### iteration step :  400  rmse :  0.017983693814787885
### iteration step :  450  rmse :  0.017300739181607964
### iteration step :  500  rmse :  0.016964143058819988
### iteration step :  550  rmse :  0.016791808565876743
### iteration step :  600  rmse :  0.01669850293306415
### iteration step :  650  rmse :  0.01664293860122892
### iteration step :  700  rmse :  0.01660455314034314
### iteration step :  750  rmse :  0.016572993477699298
### iteration step :  800  rmse :  0.01654311678640244
### iteration step :  850  rmse :  0.016512497103841783
### ite

In [7]:
pred_matrix = np.dot(P, Q.T)
print('예측 행렬:\n', np.round(pred_matrix, 3))

예측 행렬:
 [[3.991 0.897 1.306 2.002 1.663]
 [6.695 4.978 0.978 2.981 1.003]
 [6.676 0.392 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.141]]
